# Train a model Tensorflow/Keras using Amazon SageMaker

In this blog post, we’ll demonstrate how to train a model by TensorFlow/Keras using Amazon SageMaker

### Step 1. Set up

After create a new notebook instance, set the kernel to conda_tensorflow_p36.

The get_execution_role function retrieves the AWS Identity and Access Management (IAM) role you created at the time of creating your notebook instance.

In [ ]:
import boto3, re
from sagemaker import get_execution_role

role = get_execution_role()

### Step 2. Train the model

In [ ]:
%%writefile train.py

import argparse
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense

# import pandas as pd

if __name__ == '__main__':
    parser = argparse.ArgumentParser()

    parser.add_argument('--epochs', type=int, default=10)
    parser.add_argument('--batch-size', type=int, default=128)
    parser.add_argument('--learning-rate', type=float, default=0.1)
    parser.add_argument('--gpu-count', type=int, default=os.environ['SM_NUM_GPUS'])
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])

    args, _ = parser.parse_known_args()
    epochs = args.epochs
    learning_rate = args.learning_rate
    batch_size = args.batch_size
    gpu_count = args.gpu_count
    model_dir = args.model_dir

    # Load mnist from tf.keras.datasets.mnist or load data from csv
    # if size of csv is small, it can be uploaded to the notebook otherwise upload it to the s3
    # training_data = pd.read_csv('train.csv')

    mnist = tf.keras.datasets.mnist
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = x_train / 255
    x_test = x_test / 255
    y_train = keras.utils.to_categorical(y_train, 10)
    y_test = keras.utils.to_categorical(y_test, 10)

    model = Sequential()
    model.add(Flatten(input_shape=(28, 28)))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(10, activation='softmax'))

    # loss, optimizers
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.SGD(lr=learning_rate),
                  metrics=['accuracy'])

    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)

    tf.saved_model.simple_save(
        tf.keras.backend.get_session(),
        os.path.join(model_dir, '1'),
        inputs={'inputs': model.input},
        outputs={t.name: t for t in model.outputs})

In [ ]:
from sagemaker.tensorflow import TensorFlow

model = TensorFlow(
    entry_point='train.py', train_instance_type='ml.m4.xlarge',
    train_instance_count=1, role=role,
    base_job_name='mnist', framework_version='1.15',
    py_version='py3', script_mode=True
)

model.fit()

### Step 3. Deploy the trained model

#### Deploy step takes a long time, after that, sagemaker model and endpoint will be created

In [ ]:
%%time
predictor = model.deploy(initial_instance_count=1,
                                   instance_type='ml.m4.xlarge', endpoint_name='endpoint-name')

### Step 4. Invoke the endpoint

#### Invoke the SageMaker endpoint from the notebook

In [ ]:
endpoint_name = predictor.endpoint

import sagemaker
from sagemaker.tensorflow.model import TensorFlowModel

sagemaker_session = sagemaker.Session()
predictor = sagemaker.tensorflow.model.TensorFlowPredictor(endpoint_name, sagemaker_session)

import numpy as np

data = np.random.randn(1, 28, 28)

predictor.predict(data)

#### Invoke the SageMaker endpoint using a boto3 client

In [ ]:
import json
import boto3
import numpy as np
import io
 
client = boto3.client('runtime.sagemaker')

data = np.random.randn(1, 28, 28).tolist()

response = client.invoke_endpoint(EndpointName=endpoint_name, Body=json.dumps(data), ContentType='application/json')

response_body = response['Body'].read()

data = json.loads(response_body)['predictions']

value = np.argmax(data)

print(value)

### Step 5. Clean up

To avoid incurring unnecessary charges, use the AWS Management Console to delete the resources that you created for this exercise:
https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html